## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [ ]:
using Plots
using IJulia

In [ ]:
using TensorFlow
using Distributions
using StatsBase
using StaticArrays

In [ ]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

In [ ]:
"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


In [ ]:
function demonstration_plot(est, dataset, data, args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    @show typeof(data)
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

In [ ]:
function demo(dataset, layers, epochs=20_000; max_conditioning_epochs=2000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))

    condition!(est; max_epochs = max_conditioning_epochs)
    println("Conditioning Done")
    Plots.gr()
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    plotly()
    demonstration_plot(est, dataset, data) |> IJulia.display
    
    est    
end

In [ ]:
est=demo(GenerateDatasets.Likas1(), [64, 64, 256], 1_000)

In [ ]:
let 
    sess = Session(Graph())
    X=constant([1.0,2,3])
    y=Ops.mul(X,X)
    @show y
end

In [ ]:
loglikelihood(GenerateDatasets.Likas1(), [2.0])

In [ ]:
GenerateDatasets.Likas1()

In [ ]:
sort(component_weights(est))

In [ ]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

In [ ]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 10_000)

In [ ]:
demo(Arcsine(1,4), [64,64], 20_000)

In [7]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)

2×5000 Array{Float64,2}:
 0.128923  0.0275165  0.174635   0.0790673  …  0.0390223  0.125306   0.030824
 0.174406  0.103756   0.0603885  0.129334      0.0500258  0.0198605  0.020437

In [8]:
size(data,1)

2

In [9]:
@show loglikelihood(dataset, data)

loglikelihood(dataset, data) = 16094.379124341185


16094.379124341185

In [10]:
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

2017-09-20 20:08:51.522920: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:08:51.522950: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:08:51.522955: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:08:51.726509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-20 20:08:51.727014: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

pdf = <Tensor pdf:1 shape=unknown dtype=Float64>


DensityEstimationML.NeuralDensityEstimator{2}(Session(Ptr{Void} @0x00007f862844ba30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(2, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [11]:
condition!(est)

(ii, ysmin, ysmax, condition_loss) = (1, [1.00345], [1.00345], [0.993132])
(ii, ysmin, ysmax, condition_loss) = (51, [1.15976], [1.16003], [0.731074])
(ii, ysmin, ysmax, condition_loss) = (101, [1.53263], [1.53849], [0.496684])
(ii, ysmin, ysmax, condition_loss) = (151, [1.50045], [1.51411], [0.48654])
(ii, ysmin, ysmax, condition_loss) = (201, [1.50034], [1.52537], [0.475611])
(ii, ysmin, ysmax, condition_loss) = (251, [1.5003], [1.54026], [0.461667])
(ii, ysmin, ysmax, condition_loss) = (301, [1.49962], [1.55846], [0.444581])
(ii, ysmin, ysmax, condition_loss) = (351, [1.49798], [1.58027], [0.424158])
(ii, ysmin, ysmax, condition_loss) = (401, [1.49488], [1.60591], [0.40021])
(ii, ysmin, ysmax, condition_loss) = (451, [1.48969], [1.63541], [0.372719])
(ii, ysmin, ysmax, condition_loss) = (501, [1.48171], [1.66833], [0.342047])
(ii, ysmin, ysmax, condition_loss) = (551, [1.47034], [1.70364], [0.309044])
(ii, ysmin, ysmax, condition_loss) = (601, [1.45528], [1.73983], [0.274962])
(ii, 

In [ ]:

fit!(est, data'; epochs=10_000, callback=running_plot())

In [ ]:

fit!(est, data; epochs=10_000, callback=running_plot())
println("Fitting Done")

In [ ]:
sess = est.sess
run(sess, gradients(sess.graph(["yt"]), Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.3]))

In [12]:
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

2×3 Array{Float64,2}:
 1.36282  1.36282  0.565036
 1.36311  1.36311  0.565153

In [14]:
sess = est.sess
run(sess, sess.graph["numerator"], Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

2×3 Array{Float32,2}:
 1.30008  1.30008  0.539024
 1.30036  1.30036  0.539136

In [ ]:
plotly()

In [ ]:
"""
    meshgrid(xs, ys, zfun)

Evalates `zfun(x,y)` at each point in `xs`, and `ys`.
Returns 3 vectors, a list of x points a list of y points and the value of z at that point.
"""
function meshgrid(xs, ys, zfun) 
    # There is a cute generalisation of this with a `@generated` function
    xpoints = eltype(xs)[]
    ypoints = eltype(ys)[]
    zpoints = typeof(zfun(xs[1],ys[1]))[]
    sizehint!.([xpoints, ypoints, zpoints], length(xs)*length(ys))
    for x in xs, y in ys
        push!(xpoints,x)
        push!(ypoints,y)
        push!(zpoints,zfun(x,y))
    end
    
    xpoints, ypoints, zpoints
end


In [ ]:
X=-0.1:0.01:0.3
Y=-0.1:0.01:0.3
scatter3d(meshgrid(X,Y, (x,y)->pdf(est, [x,y]))...)

In [ ]:
pdf(est, SMatrix{1,2}([0.1 0.1]))

In [ ]:
est

In [ ]:
size([0.1 0.1])

In [ ]:
?SArray